<a href="https://colab.research.google.com/github/flora0110/sentence_prediction/blob/main/SBERT_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install udicOpenData

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.8/17.8 MB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for udicOpenData: filename=udicOpenData-2.4-py3-none-any.whl size=18094423 sha256=8b05f1718307f34f21cb4204f2629ef4166be4a8695be3f114343282b508f8ea
  Stored in directory: /root/.cache/pip/wheels/86/4f/46/2a28bc28485ba1f9d67dead62ac6aba14089a6cf1d5fbad365
Successfully built udicOpenData


In [ ]:
!pip install opencc-python-reimplemented

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 3.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from opencc import OpenCC
import json
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.neighbors import NearestNeighbors
import pickle
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
from udicOpenData.stopwords import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


# Data Exploration

In [ ]:
folder_path = "/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/sample_code/dataset/"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open(folder_path+'train_data.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)

with open(folder_path+'test_data.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

In [ ]:
print("===Train Dataset===")
print("Key: ", train_data[0].keys())
print("Size: ", len(train_data))

print("===Test Dataset===")
print("Key: ", test_data[0].keys())
print("Size: ", len(test_data))

===Train Dataset===
Key:  dict_keys(['id', 'reason', 'sentance', 'fact'])
Size:  500
===Test Dataset===
Key:  dict_keys(['id', 'reason', 'sentance', 'fact'])
Size:  100


In [ ]:
print(train_data[0])
print(test_data[0])

{'id': 'SlU-F40BlrB_n9qTlLQf', 'reason': '偽造文書', 'sentance': 120, 'fact': '一、陳泰龍明知其與泰國籍女子陳玉婉（泰國名字為JARUWANWONGPRATOOM，陳玉婉涉嫌本案偽造文書部分，另經臺灣臺南地方法院檢察署檢察官通緝中）間並無結婚之真意，然因詹巧薇、張晉源等人欲找臺灣男子前往泰國與泰國女子辦理假結婚，林清水負責介紹臺灣男子予詹巧薇、張晉源等人以賺取新臺幣（下同）1萬元之仲介費，適陳泰龍依林清水於報紙上刊登之徵人廣告應徵時，由林清水告知泰國女子欲以假結婚之方式申請來臺居留工作，其若擔任假結婚之人頭老公，除不用負擔前往泰國之食宿、機票費用外，尚可獲取5、6萬元之報酬，迨陳泰龍應允同意後，林清水將其轉介紹予詹巧薇、張晉源等人，陳泰龍遂與林清水、詹巧薇、張晉源等人共同基於行使使公務員登載不實公文書之概括犯意聯絡，先由陳泰龍依指示於民國93年4月3日前往泰國，並於同年月7日與陳玉婉在泰國曼谷市法拉坎農區註冊辦事處登記結婚，取得上開註冊辦事處核發之結婚登記書、結婚證書等文件後，陳泰龍即行返臺，並於同年5月19日持經我國駐泰國臺北經濟文化辦事處認證之上開結婚登記書、結婚證書等資料，至臺南縣永康市（改制前稱謂，以下沿用舊制稱謂）戶政事務所填寫結婚登記申請書，申請辦理其與陳玉婉之結婚登記，因此使上開戶政事務所僅有形式審查權之承辦公務員陷於錯誤，於形式上審核後，將陳泰龍與陳玉婉於93年4月7日結婚之不實事項，登載於其職務上所掌屬公文書之戶籍登記資料內，並據以核發戶籍謄本，而足以生損害於戶政機關對於戶政管理及戶籍登記資料維護之正確性。迨陳玉婉來臺後，為取得居留身分，再由陳泰龍、陳玉婉承前概括之犯意聯絡，於同年6月14日及94年5月5日，均持前開登載不實之戶籍謄本等資料，先後向臺南縣政府警察局外事課申請核發陳玉婉之外僑居留證及申請居留證延期而連續行使之（業經檢察官當庭補充），經承辦人員實質審查結果，均未發覺陳泰龍與陳玉婉假結婚之實情，而發給陳玉婉居留證或核准延期，各足以生損害於主管機關對外僑居留管理之正確性。二、案經臺南市警察局（現改制為臺南市政府警察局，以下沿用舊制稱謂）第一分局報告臺灣臺南地方法院檢察署檢察官偵查起訴。理由甲、程序方面：一、本件以下所引用具傳聞證據性質之供述證據，因本件被告陳泰龍、檢察官等

In [ ]:
train_fact_len = []
test_fact_len = []

for data in train_data:
    train_fact_len.append(len(data['fact']))

for data in test_data:
    test_fact_len.append(len(data['fact']))


In [ ]:
print("===Train Dataset===")
print("Length: ", len(train_fact_len))
print("Max Length: ", max(train_fact_len))
print("Min Length: ", min(train_fact_len))
print("Avg. Length: ", sum(train_fact_len) / len(train_fact_len))

print("===Test Dataset===")
print("Length: ", len(test_fact_len))
print("Max Length: ", max(test_fact_len))
print("Min Length: ", min(test_fact_len))
print("Avg. Length: ", sum(test_fact_len) / len(test_fact_len))

===Train Dataset===
Length:  500
Max Length:  19745
Min Length:  328
Avg. Length:  7933.656
===Test Dataset===
Length:  100
Max Length:  19277
Min Length:  280
Avg. Length:  8231.62


In [ ]:
def extract_key_contents(data):
    facts = []
    sentences = []
    for item in data:
        facts.append(item.get('fact'))
        sentences.append(item.get('sentance'))
    return facts, sentences

In [ ]:
train_facts, train_sentences = extract_key_contents(train_data)
test_facts, test_sentences = extract_key_contents(test_data)

In [ ]:
print("Train facts:", train_facts[:5])
print("Train Sentences:", train_sentences[:5])
print("Test facts:", test_facts[:5])
print("Test Sentences:", test_sentences[:5])

Train facts: ['一、陳泰龍明知其與泰國籍女子陳玉婉（泰國名字為JARUWANWONGPRATOOM，陳玉婉涉嫌本案偽造文書部分，另經臺灣臺南地方法院檢察署檢察官通緝中）間並無結婚之真意，然因詹巧薇、張晉源等人欲找臺灣男子前往泰國與泰國女子辦理假結婚，林清水負責介紹臺灣男子予詹巧薇、張晉源等人以賺取新臺幣（下同）1萬元之仲介費，適陳泰龍依林清水於報紙上刊登之徵人廣告應徵時，由林清水告知泰國女子欲以假結婚之方式申請來臺居留工作，其若擔任假結婚之人頭老公，除不用負擔前往泰國之食宿、機票費用外，尚可獲取5、6萬元之報酬，迨陳泰龍應允同意後，林清水將其轉介紹予詹巧薇、張晉源等人，陳泰龍遂與林清水、詹巧薇、張晉源等人共同基於行使使公務員登載不實公文書之概括犯意聯絡，先由陳泰龍依指示於民國93年4月3日前往泰國，並於同年月7日與陳玉婉在泰國曼谷市法拉坎農區註冊辦事處登記結婚，取得上開註冊辦事處核發之結婚登記書、結婚證書等文件後，陳泰龍即行返臺，並於同年5月19日持經我國駐泰國臺北經濟文化辦事處認證之上開結婚登記書、結婚證書等資料，至臺南縣永康市（改制前稱謂，以下沿用舊制稱謂）戶政事務所填寫結婚登記申請書，申請辦理其與陳玉婉之結婚登記，因此使上開戶政事務所僅有形式審查權之承辦公務員陷於錯誤，於形式上審核後，將陳泰龍與陳玉婉於93年4月7日結婚之不實事項，登載於其職務上所掌屬公文書之戶籍登記資料內，並據以核發戶籍謄本，而足以生損害於戶政機關對於戶政管理及戶籍登記資料維護之正確性。迨陳玉婉來臺後，為取得居留身分，再由陳泰龍、陳玉婉承前概括之犯意聯絡，於同年6月14日及94年5月5日，均持前開登載不實之戶籍謄本等資料，先後向臺南縣政府警察局外事課申請核發陳玉婉之外僑居留證及申請居留證延期而連續行使之（業經檢察官當庭補充），經承辦人員實質審查結果，均未發覺陳泰龍與陳玉婉假結婚之實情，而發給陳玉婉居留證或核准延期，各足以生損害於主管機關對外僑居留管理之正確性。二、案經臺南市警察局（現改制為臺南市政府警察局，以下沿用舊制稱謂）第一分局報告臺灣臺南地方法院檢察署檢察官偵查起訴。理由甲、程序方面：一、本件以下所引用具傳聞證據性質之供述證據，因本件被告陳泰龍、檢察官等人於本院準備程序進行時，均未爭執前開陳述之證據能力，且迄至本案言詞辯論終結前亦均未聲明異議，應視為已同意援引作為證據，又

In [ ]:
def read_string_array_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            string_array = file.readlines()
            # 去除每個元素末尾的換行符
            string_array = [string.strip() for string in string_array]
        print(f"String array read from {file_path} successfully.")
        return string_array
    except Exception as e:
        print(f"Error occurred while reading string array: {e}")
        return None

In [ ]:
train_summaries = read_string_array_from_file(folder_path + 'train_summaries.txt')
test_summaries = read_string_array_from_file(folder_path + 'test_summaries.txt')

String array read from /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/sample_code/dataset/train_summaries.txt successfully.
String array read from /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/sample_code/dataset/test_summaries.txt successfully.


In [ ]:
train_summaries_len = []
test_summaries_len = []

for data in train_summaries:
    train_summaries_len.append(len(data))

for data in test_summaries:
    test_summaries_len.append(len(data))

In [ ]:
print("===Train Dataset===")
print("Length: ", len(train_summaries_len))
print("Max Length: ", max(train_summaries_len))
print("Min Length: ", min(train_summaries_len))
print("Avg. Length: ", sum(train_summaries_len) / len(train_summaries_len))

print("===Test Dataset===")
print("Length: ", len(test_summaries_len))
print("Max Length: ", max(test_summaries_len))
print("Min Length: ", min(test_summaries_len))
print("Avg. Length: ", sum(test_summaries_len) / len(test_summaries_len))

===Train Dataset===
Length:  500
Max Length:  1291
Min Length:  0
Avg. Length:  388.652
===Test Dataset===
Length:  100
Max Length:  1219
Min Length:  97
Avg. Length:  394.01


In [ ]:
print("Train summaries:", train_summaries[:5])
print("Test summaries:", test_summaries[:5])

Train summaries: ['被告明知與泰國籍女子陳玉婉（JARUWANWONGPRATOOM）間並無結婚之真意，仍與林清水、詹巧薇、張晉源等人共同基於行使使公務員登載不實公文書之概括犯意聯絡，辦理假結婚手續。被告與陳玉婉在泰國辦理結婚手續後，返回臺灣辦理結婚登記，使承辦公務員陷於錯誤，登載不實事項於戶籍登記資料內，核發戶籍謄本。陳玉婉來臺後，為取得居留身分，被告與她共同向臺南縣政府警察局外事課申請核發居留證及延期，承辦人員實質審查結果未發覺假結婚，而核發居留證或核准延期。', '甲○○於民國100年11月1日上午7時許，在臺南市北安橋路旁拾得可發射子彈具有殺傷力之槍枝（槍枝管制編號：0000000000號）後持有之，並於100年11月6日主動向員警坦承犯行而自首接受裁判甲○○於本院審理該案（100年度訴字第1635號）時編造其母楊陳敏於100年11月1日經急診住院接受治療，其係為照顧住院之母親楊陳敏才遲延報繳槍枝之情節，辯稱其無持有可發射子彈具有殺傷力槍枝之犯意甲○○為取信於法院乃基於行使偽造私文書之犯意，於100年11月6日至101年1月12日間某日在不詳地點，偽刻奇美醫療財團法人奇美醫院（下稱奇美醫院）、院長邱仲慶及醫師甲○○之印章各1顆，再偽造如附表編號四所示診斷證明書（下稱系爭診斷證明書）提出於本院（收件日期為101年1月12日），表示系爭診斷證明書記載之內容業經認證屬實之意，足以生損害於奇美醫院、邱仲慶、甲○○及國家司法權行使之正確性', '及理由一、犯罪事實：白秋霖與泰國籍女子WANGMONTRIDOKMAI（中文姓名白雪蘭，另為職權不起訴處分）均明知並無結婚之真意，為使WANGMONTRIDOKMAI得以結婚方式居留在臺灣地區工作，共同基於使公務員登載不實文書之概括犯意聯絡，於民國93年10月29日，至泰國曼谷，辦理假結婚手續，並在泰國取得不實之結婚證明書等相關資料，由白秋霖於同年11月18日，前往臺中縣沙鹿鎮戶政事務所，持上開結婚證件，申請辦理結婚戶籍登記，使該戶政事務所不知情之承辦人員將白秋霖與WANGMONTRIDOKMAI間虛偽辦理結婚之不實事項，登載於職務上掌管之戶籍登記資料之電子資訊檔案紀錄、戶籍謄本等公文書，足以生損害於戶政機關對於戶籍登記事項管理之正確性白秋霖明知上開婚姻係虛偽婚姻，於94年3月30日具狀向本院訴請離婚，經本院

# Preprocessing

## 斷詞

In [ ]:
cc = OpenCC('t2s')

In [ ]:
# 分詞加空白
def split_passage(passage):
    word_and_tag = list(rmsw(passage, flag=True))
    _temp = list(filter(lambda x: x[1]!='nr' and x[1]!='ns', word_and_tag))
    text = " ".join(list(map(lambda x:x[0], _temp)))
    return text

In [ ]:
# tokenized_train_facts = []

# for data in train_facts:
#     tokenized_train_facts.append(split_passage(cc.convert(data)))

In [ ]:
# tokenized_test_facts = []

# for data in test_facts:
#     tokenized_test_facts.append(split_passage(cc.convert(data)))

In [ ]:
# tokenized_train_facts[0]

'明知 与 籍 女子 名字 为 JARUWANWONGPRATOOM 涉嫌 本案 伪造文书 部分 经 地方法院 检察署 检察官 通缉 中 间 并 无 结婚 真意 源等 人 欲 找 男子 前往 与 女子 办理 假结婚 林 清水 负责 介绍 男子 予 源等 人 赚取 新台币 下同 万元 仲介 费 适 清水 于 报纸 刊登 征 人 广告 应征 时 清水 告知 女子 欲 假结婚 方式 申请 来 台 居留 工作 其若 担任 假结婚 人头 不用 负担 前往 食宿 机票 费用 外 尚可 获取 万元 报酬 迨 应允 同意 后 林 清水 将 转 介绍 予 源等 人 遂 与 清水 源等 人 共同 基于 行使 使 公务员 登载 不实 公文书 概括 犯意 联络 先 指示 于 民国 前往 并 于 同年 月 日 与 法拉 坎农 区 注册 办事处 登记 结婚 取得 上开 注册 办事处 核发 结婚登记 书 结婚证书 文件 后 即行 返台 并 于 同年 持 经 我国 驻 台北 经济 文化 办事处 认证 之上 开 结婚登记 书 结婚证书 资料 永康市 改制 前 称谓 以下 沿用 旧制 称谓 户政事务 填写 结婚登记 申请 办理 与 结婚登记 使 开 户政事务 所仅 有形 式 审查 承办 公务员 陷于 错误 于 形式 审核 后 将 与 于 结婚 不实 事项 登载 于 职务 上所 掌属 公文书 户籍 登记 资料 内 并 据 核发 户籍誊本 足以 生 损害 于 户政 机关 对于 户政 管理 户籍 登记 资料 维护 正确性 迨 来 台后 为 取得 居留 身分 前 概括 犯意 联络 于 同年 持 前 开 登载 不实 户籍誊本 资料 先后 县政府 警察局 外事 课 申请 核发 外侨 申请 延期 连续 行使 业经 检察官 当庭 补充 经 承办 人员 实质 审查 结果 未 发觉 与 假结婚 实情 发给 核准 延期 足以 生 损害 于 主管机关 对 外侨 居留 管理 正确性 案经 警察局 现 改制 为 市政府 警察局 以下 沿用 旧制 称谓 第一 分局 报告 地方法院 检察署 检察官 侦查 起诉 理由 甲 程序 方面 本件 以下 引用 具 传闻 证据 性质 供述 证据 本件 被告 检察官 人 于 本院 准备 程序 进行 时 未 争执 前 开 证据 能力 迄至 本案 言词辩论 终结 前 亦 未声明 异议 应 视为 已 同

In [ ]:
# tokenized_test_facts[0]

'缘 所涉 行使 伪造 私 文书 犯行 业经 本院 年度 诉 字 号 判决 处 有期徒刑 缓刑 确定 需款 使用 知悉 借款 管道 为求 顺利 借款 人 共同 意图 为 不法 所有 基于 行使 伪造 私 文书 诈欺 取财 犯意 联络 知悉 车牌号码 号 营业 小客车 系 光向 交通 有限公司 下称 志 公司 租赁 车辆 先于 不详 时间 不详 地点 伪刻 交通 有限公司 负责人 印章 正确 姓名 为 误刻 为 枚 并 伪造 代表 公司 与 签署 计程车 客运 驾驶 人 自备 车辆 参与 经营 契约书 下称 本案 参与 经营 契约书 虚伪 记载 提供 上述 营业 小客车 靠行 公司 营业 不实 内容 并持 上开 伪刻 印章 伪造 交通 有限公司 印文 枚 印文 枚 于 本案 参与 经营 契约书 于 交 约人 乙方 栏 签名 人 复于 同日 协同 不知情 配偶 所涉 行使 伪造 私 文书 犯行 业经 本院 年度 诉 字 号 判决 无罪 确定 前往 区 街 巷 号 楼 经营 宏帝 当 舖 现 更名 为 渣打 当 舖 交付 伪造 本案 参与 经营 契约书 当铺 借款 新台币 下同 万元 以为 行使 并 约定 担任 连带保证 人 误信 上开 车辆 为 所有 陷入 错误 于 扣除 利息 后 交付 万 千元 借款 予 夫妇 足以 生 损害 公司 嗣 仅 清偿 每期 千元 分期 款 期计 万 千元 未 全额 清偿 无力 负担 债务 生始 查知 上开 车辆 非 所有 悉上 情 案经 生诉 士林 地方法院 检察署 检察官 侦查 起诉 理由 证据 能力 部分 被告 以外 之人 于 审判 外 言词 书面 法律 规定 外 作为 证据 被告 以外 之人 于 审判 外 虽 不符 合同法 条 条 规定 经 当事人 于 审判程序 同意 作为 证据 法院 审酌 该 言词 书面 作成 时 情况 认为 适当 亦 为 证据 刑事诉讼法 条 项 及第 条 项 分别 定有 明文 经查 判决 引用 该 被告 以外 之人 于 审判 外 虽 属 传闻 证据 当事人 于 本院 审判 期日 中均 表示同意 作为 证据 方法 不予 争执 迄至 言词辩论 终结 前 亦 未再 声明 异议 见 本院 卷 页 页 背面 本院 审酌 上开 证据 资料 制作 时 情况 尚无 违法 不当 证明 力 明显 过 低 瑕疵 亦 认为 以之 作

In [ ]:
# def save_string_array_to_file(string_array, file_path):
#     try:
#         with open(file_path, 'w') as file:
#             for string in string_array:
#                 file.write(string + '\n')
#         print(f"String array saved to {file_path} successfully.")
#     except Exception as e:
#         print(f"Error occurred while saving string array: {e}")

In [ ]:
# save_string_array_to_file(tokenized_train_facts, folder_path + 'tokenized_train_facts.txt')
# save_string_array_to_file(tokenized_test_facts, folder_path + 'tokenized_test_facts.txt')

String array saved to /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/dataset_new/tokenized_train_facts.txt successfully.
String array saved to /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/dataset_new/tokenized_test_facts.txt successfully.


In [ ]:
tokenized_train_summaries = read_string_array_from_file(folder_path + 'tokenized_train_summaries.txt')
tokenized_test_summaries = read_string_array_from_file(folder_path + 'tokenized_test_summaries.txt')

String array read from /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/sample_code/dataset/tokenized_train_summaries.txt successfully.
String array read from /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/sample_code/dataset/tokenized_test_summaries.txt successfully.


## Data embedding

In [ ]:
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

### Model test

In [ ]:
wordpairs = [['老師', '教師', '泰國'],
             ['商品', '貨物', '跑步']]

for wordpair in wordpairs:
    embeddings = model.encode(wordpair)
    print(wordpair[0], 'vs',  wordpair[1], 'distance =', util.pytorch_cos_sim(embeddings[0], embeddings[1]))
    print(wordpair[0], 'vs',  wordpair[2], 'distance =', util.pytorch_cos_sim(embeddings[0], embeddings[2]))
    print(wordpair[1], 'vs',  wordpair[2], 'distance =', util.pytorch_cos_sim(embeddings[1], embeddings[2]))

老師 vs 教師 distance = tensor([[0.9929]])
老師 vs 泰國 distance = tensor([[0.3189]])
教師 vs 泰國 distance = tensor([[0.3134]])
商品 vs 貨物 distance = tensor([[0.6787]])
商品 vs 跑步 distance = tensor([[0.4246]])
貨物 vs 跑步 distance = tensor([[0.4545]])


In [ ]:
wordpairs = [['老师', '教师', '泰国'],
              ['商品', '货物', '跑步']]

for wordpair in wordpairs:
    embeddings = model.encode(wordpair)
    print(wordpair[0], 'vs',  wordpair[1], 'distance =', util.pytorch_cos_sim(embeddings[0], embeddings[1]))
    print(wordpair[0], 'vs',  wordpair[2], 'distance =', util.pytorch_cos_sim(embeddings[0], embeddings[2]))
    print(wordpair[1], 'vs',  wordpair[2], 'distance =', util.pytorch_cos_sim(embeddings[1], embeddings[2]))

老师 vs 教师 distance = tensor([[0.9646]])
老师 vs 泰国 distance = tensor([[0.3492]])
教师 vs 泰国 distance = tensor([[0.3385]])
商品 vs 货物 distance = tensor([[0.9299]])
商品 vs 跑步 distance = tensor([[0.4246]])
货物 vs 跑步 distance = tensor([[0.4699]])


### Data embedding

In [ ]:
# train_embeddings = model.encode(tokenized_train_summaries)

In [ ]:
# test_embeddings = model.encode(tokenized_test_summaries)

In [ ]:
# np.savetxt(folder_path + 'train_summaries_embeddings.txt', train_embeddings)
# np.savetxt(folder_path + 'test_summaries_embeddings.txt', test_embeddings)

In [ ]:
train_embeddings = np.loadtxt(folder_path + 'train_summaries_embeddings.txt')
test_embeddings = np.loadtxt(folder_path + 'test_summaries_embeddings.txt')

# KNN

In [ ]:
k = 5
knn_model_SBERT = NearestNeighbors(n_neighbors=k, metric='cosine')
knn_model_SBERT.fit(train_embeddings)

NearestNeighbors(metric='cosine')

In [ ]:
# 將模型保存到文件
with open(folder_path + 'knn_model_SBERT.pkl', 'wb') as f:
    pickle.dump(knn_model_SBERT, f)

In [ ]:
# 從文件加載模型
with open(folder_path + 'knn_model_SBERT.pkl', 'rb') as f:
    knn_model_SBERT_loaded = pickle.load(f)

## 以第一篇test fact進行測試

In [ ]:
distances, indices = knn_model_SBERT_loaded.kneighbors(test_embeddings[0].reshape(1, -1))

In [ ]:
indices

array([[143, 415,  47,  61, 250]])

In [ ]:
distances

array([[0.26868572, 0.28303525, 0.29018348, 0.29428214, 0.29455762]])

In [ ]:
test_facts[0]

'○、緣徐鴻光（所涉行使偽造私文書犯行，業經本院以95年度訴字第955號判決處有期徒刑3月、緩刑3年確定）因需款使用，知悉穆立明有借款管道，為求順利借款，2人竟共同意圖為自己不法所有，基於行使偽造私文書及詐欺取財之犯意聯絡，知悉車牌號碼00-000號營業小客車係徐鴻光向志朗交通有限公司（下稱志朗公司）租賃之車輛，竟由穆立明先於不詳時間、不詳地點，偽刻「志朗交通有限公司」及負責人「陳健志」印章（正確姓名為陳建志，誤刻為「陳健志」）各1枚，並偽造「陳健志」代表志朗公司與徐鴻光簽署之「臺北市計程車客運駕駛人自備車輛參與經營契約書」（下稱本案參與經營契約書），虛偽記載徐鴻光提供上述營業小客車靠行志朗公司營業等不實內容，並持上開偽刻印章，偽造「志朗交通有限公司」印文4枚、「陳健志」印文6枚於本案參與經營契約書上，再於92年3月6日交徐鴻光於立約人乙方欄簽名，2人復於同日協同不知情之徐鴻光配偶徐樂仁（所涉行使偽造私文書犯行，業經本院以95年度訴字第955號判決無罪確定），前往臺北市○○區○○街00巷0號2樓王傑生所經營之宏帝當舖（現更名為渣打當舖），由徐鴻光、穆立明交付偽造本案參與經營契約書向宏帝當鋪借款新臺幣（下同）10萬元以為行使，並約定由徐樂仁擔任連帶保證人，王傑生誤信上開車輛為徐鴻光所有而陷入錯誤，於扣除利息後交付9萬1千元借款予徐鴻光夫婦，足以生損害於志朗公司、陳建志、王傑生。嗣因徐鴻光僅清償每期7千元之分期款共11期計7萬7千元，未全額清償即無力負擔債務，王傑生始查知上開車輛非徐鴻光所有，而悉上情。二、案經王傑生訴由臺灣士林地方法院檢察署檢察官偵查起訴。理由○、證據能力部分：被告以外之人於審判外之言詞或書面陳述，除法律有規定者外，不得作為證據；又被告以外之人於審判外之陳述，雖不符合同法第159條之1至第159條之4之規定，但經當事人於審判程序同意作為證據，法院審酌該言詞陳述或書面陳述作成時之情況，認為適當者，亦得為證據。刑事訴訟法第159條第1項及第159條之5第1項分別定有明文。經查，本判決所引用之各該被告以外之人於審判外之陳述，雖屬傳聞證據，然當事人於本院審判期日中均表示同意作為證據方法而不予爭執，且迄至言詞辯論終結前亦未再聲明異議（見本院卷第68頁至第68頁背面），本院審酌上開證據資料製作時之情況，尚無違法不當及證明力明顯過低之瑕疵，亦認為以之作為證據應屬適當

In [ ]:
# 打印最相近的五篇文章的索引及距離
for i, index in enumerate(indices[0]):
    print("Article Index:", index, "Distance:", distances[0][i])
    print("Article Text:", train_facts[index])

Article Index: 143 Distance: 0.26868572455441253
Article Text: 一、己○○（原名蔡佳伶，於民國89年3月20日更名），於88年12月下旬，因亟需貸款，經洽代辦貸款業者自稱「陳國治」之成年男子協助。「陳國治」與甲○○（於另案判決確定）熟識，前經「陳國治」向其表示要借車辦貸款，意即當時有銀行憑借款人名下有出廠5年內之汽車，即可提供個人小額信用貸款，故「陳國治」需有此等汽車過戶至借款人名下，僅製造財產證明，毋庸設定抵押，車主原車可用，且過戶後該車稅金即由「陳國治」代繳，如代辦之貸款獲銀行核撥，「陳國治」可抽成充作佣金，並分佣給甲○○；甲○○欣然同意，適其配偶丙○○名下車牌號碼00-0000號自用小客車（下稱系爭汽車）符合條件，甲○○遂藉口買賣使不知情之丙○○於88年12月18日至交通部公路局臺中區監理所（下稱臺中區監理所）將系爭汽車過戶至不知情之戊○○名下，既無讓與合意，真正車主仍為丙○○，實際上由甲○○用車。於己○○委由「陳國治」代辦貸款後，「陳國治」與甲○○依計畫要將已在戊○○名下之系爭汽車過戶至己○○名下，己○○誤以系爭汽車為戊○○所有，明知自己並無受讓系爭汽車所有權之事實，為求辦貸款，竟依「陳國治」與甲○○之指示，而基於使公務員登載不實之概括犯意聯絡，於88年12月20日上午10時51分許，向臺中區監理所偽以辦理過戶，在該汽車車輛異動登記書上之「新車主簽章」欄，簽寫姓名「蔡佳伶」，使不知情之公務員於職務上所掌之公文書（含輸入電腦形成電磁記錄之準公文書）即車籍登記資料上登載新車主為「蔡佳伶」，並據以補發行車執照，足以生損害於公路監理機關對於車籍管理之正確性及丙○○。己○○嗣依「陳國治」與甲○○指示，明知乙○○亦係求辦貸款，並無受讓系爭汽車所有權之事實，仍承上概括犯意聯絡，於89年4月20日下午1時38分許，夥同乙○○再向臺中區監理所偽以辦理過戶，在該汽車車輛異動登記書上之「原車主簽章」欄，簽寫姓名「己○○」，使不知情之公務員於職務上所掌之公文書（含輸入電腦形成電磁記錄之準公文書）即車籍登記資料上登載新車主為乙○○，並據以補發行車執照，足以生損害於公路監理機關對於車籍管理之正確性及丙○○。嗣於89年5月10日上午11時51分許，由甲○○帶同乙○○本人向臺中區監理所偽以辦理過戶予戊○○（己○○不知情），經戊○

## 使用全部test facts在train facts中尋找最相近的五篇facts並計算預測刑期

In [ ]:
predicted_sentences_all = []

for test_embedding in test_embeddings:
    test_embedding = test_embedding.reshape(1, -1)
    distances, indices = knn_model_SBERT_loaded.kneighbors(test_embedding)
    # 取得最相近的五篇文章的索引及距離
    nearest_articles_indices = indices[0]
    nearest_articles_distances = distances[0]
    predicted_sentences = 0
    # 將最相近的五篇文章的刑期加總，並計算平均值作為預測刑期
    for i, index in enumerate(nearest_articles_indices):
        predicted_sentences += train_sentences[index]
    predicted_sentence = float(predicted_sentences) / 5

    predicted_sentences_all.append(predicted_sentence)

In [ ]:
print(predicted_sentences_all)
print(test_sentences)

[111.0, 124.6, 120.0, 181.0, 215.0, 351.0, 184.0, 223.0, 256.0, 174.0, 241.0, 250.0, 168.0, 138.0, 132.0, 287.0, 129.0, 120.0, 159.0, 191.0, 205.0, 102.0, 144.0, 201.0, 167.0, 144.0, 116.0, 112.0, 217.0, 80.0, 266.0, 91.6, 217.0, 90.0, 150.0, 184.6, 163.0, 219.0, 187.0, 114.0, 94.0, 199.0, 144.0, 338.0, 132.0, 156.0, 233.0, 150.0, 135.4, 168.0, 485.2, 217.0, 177.0, 223.0, 93.0, 113.0, 553.2, 126.0, 102.0, 175.0, 165.0, 129.0, 312.0, 308.0, 103.6, 180.0, 266.0, 168.0, 217.0, 259.0, 192.0, 83.0, 163.0, 411.0, 72.0, 271.0, 168.0, 146.0, 132.0, 108.0, 91.0, 266.0, 211.0, 229.0, 123.0, 161.4, 100.0, 186.0, 138.0, 106.0, 162.0, 162.0, 121.8, 234.2, 126.0, 505.2, 302.0, 118.0, 225.0, 122.0]
[150, 242, 60, 302, 365, 150, 90, 120, 120, 180, 120, 90, 120, 90, 90, 90, 180, 270, 425, 50, 150, 90, 365, 195, 150, 50, 345, 150, 150, 90, 425, 135, 120, 240, 20, 240, 60, 120, 60, 150, 90, 240, 365, 210, 150, 120, 32, 150, 505, 90, 90, 120, 120, 120, 165, 180, 302, 90, 120, 730, 240, 40, 180, 90, 90, 18

In [ ]:
mse_linear = mean_squared_error(test_sentences, predicted_sentences_all)
rmse_linear = mean_squared_error(test_sentences, predicted_sentences_all, squared=False)
mae_linear = mean_absolute_error(test_sentences, predicted_sentences_all)

In [ ]:
print('\n==================================================')
print("Linear Regression - MSE:", mse_linear)
print("Linear Regression - RMSE:", rmse_linear)
print("Linear Regression - MAE:", mae_linear)


Linear Regression - MSE: 22384.279599999998
Linear Regression - RMSE: 149.61376808302103
Linear Regression - MAE: 104.57400000000001


# 不同 K 測試

## k=10

In [ ]:
k = 10
knn_model_SBERT = NearestNeighbors(n_neighbors=k, metric='cosine')
knn_model_SBERT.fit(train_embeddings)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [ ]:
predicted_sentences_all = []

for test_embedding in test_embeddings:
    test_embedding = test_embedding.reshape(1, -1)
    distances, indices = knn_model_SBERT.kneighbors(test_embedding)
    # 取得最相近的五篇文章的索引及距離
    nearest_articles_indices = indices[0]
    nearest_articles_distances = distances[0]
    predicted_sentences = 0
    # 將最相近的五篇文章的刑期加總，並計算平均值作為預測刑期
    for i, index in enumerate(nearest_articles_indices):
        predicted_sentences += train_sentences[index]
    predicted_sentence = float(predicted_sentences) / k

    predicted_sentences_all.append(predicted_sentence)

In [ ]:
mse_linear = mean_squared_error(test_sentences, predicted_sentences_all)
rmse_linear = mean_squared_error(test_sentences, predicted_sentences_all, squared=False)
mae_linear = mean_absolute_error(test_sentences, predicted_sentences_all)
print('\n==================================================')
print("Linear Regression - MSE:", mse_linear)
print("Linear Regression - RMSE:", rmse_linear)
print("Linear Regression - MAE:", mae_linear)


Linear Regression - MSE: 17854.171299999998
Linear Regression - RMSE: 133.61950194488827
Linear Regression - MAE: 96.12899999999999


## k=20

In [ ]:
k = 20
knn_model_SBERT = NearestNeighbors(n_neighbors=k, metric='cosine')
knn_model_SBERT.fit(train_embeddings)

NearestNeighbors(metric='cosine', n_neighbors=20)

In [ ]:
predicted_sentences_all = []

for test_embedding in test_embeddings:
    test_embedding = test_embedding.reshape(1, -1)
    distances, indices = knn_model_SBERT.kneighbors(test_embedding)
    # 取得最相近的五篇文章的索引及距離
    nearest_articles_indices = indices[0]
    nearest_articles_distances = distances[0]
    predicted_sentences = 0
    # 將最相近的五篇文章的刑期加總，並計算平均值作為預測刑期
    for i, index in enumerate(nearest_articles_indices):
        predicted_sentences += train_sentences[index]
    predicted_sentence = float(predicted_sentences) / k

    predicted_sentences_all.append(predicted_sentence)

In [ ]:
mse_linear = mean_squared_error(test_sentences, predicted_sentences_all)
rmse_linear = mean_squared_error(test_sentences, predicted_sentences_all, squared=False)
mae_linear = mean_absolute_error(test_sentences, predicted_sentences_all)
print('\n==================================================')
print("Linear Regression - MSE:", mse_linear)
print("Linear Regression - RMSE:", rmse_linear)
print("Linear Regression - MAE:", mae_linear)


Linear Regression - MSE: 16835.22775
Linear Regression - RMSE: 129.75063679997874
Linear Regression - MAE: 92.29399999999998


## k = 50

In [ ]:
k = 50
knn_model_SBERT = NearestNeighbors(n_neighbors=k, metric='cosine')
knn_model_SBERT.fit(train_embeddings)

NearestNeighbors(metric='cosine', n_neighbors=50)

In [ ]:
predicted_sentences_all = []

for test_embedding in test_embeddings:
    test_embedding = test_embedding.reshape(1, -1)
    distances, indices = knn_model_SBERT.kneighbors(test_embedding)
    # 取得最相近的五篇文章的索引及距離
    nearest_articles_indices = indices[0]
    nearest_articles_distances = distances[0]
    predicted_sentences = 0
    # 將最相近的五篇文章的刑期加總，並計算平均值作為預測刑期
    for i, index in enumerate(nearest_articles_indices):
        predicted_sentences += train_sentences[index]
    predicted_sentence = float(predicted_sentences) / k

    predicted_sentences_all.append(predicted_sentence)

In [ ]:
mse_linear = mean_squared_error(test_sentences, predicted_sentences_all)
rmse_linear = mean_squared_error(test_sentences, predicted_sentences_all, squared=False)
mae_linear = mean_absolute_error(test_sentences, predicted_sentences_all)
print('\n==================================================')
print("Linear Regression - MSE:", mse_linear)
print("Linear Regression - RMSE:", rmse_linear)
print("Linear Regression - MAE:", mae_linear)


Linear Regression - MSE: 16769.644076000004
Linear Regression - RMSE: 129.49766050396434
Linear Regression - MAE: 89.24499999999998


## k = 100

In [ ]:
k = 100
knn_model_SBERT = NearestNeighbors(n_neighbors=k, metric='cosine')
knn_model_SBERT.fit(train_embeddings)

NearestNeighbors(metric='cosine', n_neighbors=100)

In [ ]:
predicted_sentences_all = []

for test_embedding in test_embeddings:
    test_embedding = test_embedding.reshape(1, -1)
    distances, indices = knn_model_SBERT.kneighbors(test_embedding)
    # 取得最相近的五篇文章的索引及距離
    nearest_articles_indices = indices[0]
    nearest_articles_distances = distances[0]
    predicted_sentences = 0
    # 將最相近的五篇文章的刑期加總，並計算平均值作為預測刑期
    for i, index in enumerate(nearest_articles_indices):
        predicted_sentences += train_sentences[index]
    predicted_sentence = float(predicted_sentences) / k

    predicted_sentences_all.append(predicted_sentence)

In [ ]:
mse_linear = mean_squared_error(test_sentences, predicted_sentences_all)
rmse_linear = mean_squared_error(test_sentences, predicted_sentences_all, squared=False)
mae_linear = mean_absolute_error(test_sentences, predicted_sentences_all)
print('\n==================================================')
print("Linear Regression - MSE:", mse_linear)
print("Linear Regression - RMSE:", rmse_linear)
print("Linear Regression - MAE:", mae_linear)


Linear Regression - MSE: 16467.086894000004
Linear Regression - RMSE: 128.32414774312747
Linear Regression - MAE: 87.95340000000002


## k = 150

In [ ]:
k = 150
knn_model_SBERT = NearestNeighbors(n_neighbors=k, metric='cosine')
knn_model_SBERT.fit(train_embeddings)

NearestNeighbors(metric='cosine', n_neighbors=150)

In [ ]:
predicted_sentences_all = []

for test_embedding in test_embeddings:
    test_embedding = test_embedding.reshape(1, -1)
    distances, indices = knn_model_SBERT.kneighbors(test_embedding)
    # 取得最相近的五篇文章的索引及距離
    nearest_articles_indices = indices[0]
    nearest_articles_distances = distances[0]
    predicted_sentences = 0
    # 將最相近的五篇文章的刑期加總，並計算平均值作為預測刑期
    for i, index in enumerate(nearest_articles_indices):
        predicted_sentences += train_sentences[index]
    predicted_sentence = float(predicted_sentences) / k

    predicted_sentences_all.append(predicted_sentence)

In [ ]:
mse_linear = mean_squared_error(test_sentences, predicted_sentences_all)
rmse_linear = mean_squared_error(test_sentences, predicted_sentences_all, squared=False)
mae_linear = mean_absolute_error(test_sentences, predicted_sentences_all)
print('\n==================================================')
print("Linear Regression - MSE:", mse_linear)
print("Linear Regression - RMSE:", rmse_linear)
print("Linear Regression - MAE:", mae_linear)


Linear Regression - MSE: 16307.010348000003
Linear Regression - RMSE: 127.69890503837534
Linear Regression - MAE: 86.34446666666669


## k = 200 (最低)

In [ ]:
k = 200
knn_model_SBERT = NearestNeighbors(n_neighbors=k, metric='cosine')
knn_model_SBERT.fit(train_embeddings)

NearestNeighbors(metric='cosine', n_neighbors=200)

In [ ]:
predicted_sentences_all = []

for test_embedding in test_embeddings:
    test_embedding = test_embedding.reshape(1, -1)
    distances, indices = knn_model_SBERT.kneighbors(test_embedding)
    # 取得最相近的五篇文章的索引及距離
    nearest_articles_indices = indices[0]
    nearest_articles_distances = distances[0]
    predicted_sentences = 0
    # 將最相近的五篇文章的刑期加總，並計算平均值作為預測刑期
    for i, index in enumerate(nearest_articles_indices):
        predicted_sentences += train_sentences[index]
    predicted_sentence = float(predicted_sentences) / k

    predicted_sentences_all.append(predicted_sentence)

In [ ]:
mse_linear = mean_squared_error(test_sentences, predicted_sentences_all)
rmse_linear = mean_squared_error(test_sentences, predicted_sentences_all, squared=False)
mae_linear = mean_absolute_error(test_sentences, predicted_sentences_all)
print('\n==================================================')
print("Linear Regression - MSE:", mse_linear)
print("Linear Regression - RMSE:", rmse_linear)
print("Linear Regression - MAE:", mae_linear)


Linear Regression - MSE: 16147.333150500002
Linear Regression - RMSE: 127.07215725917304
Linear Regression - MAE: 86.0693


## k = 250

In [ ]:
k = 250
knn_model_SBERT = NearestNeighbors(n_neighbors=k, metric='cosine')
knn_model_SBERT.fit(train_embeddings)

NearestNeighbors(metric='cosine', n_neighbors=250)

In [ ]:
predicted_sentences_all = []

for test_embedding in test_embeddings:
    test_embedding = test_embedding.reshape(1, -1)
    distances, indices = knn_model_SBERT.kneighbors(test_embedding)
    # 取得最相近的五篇文章的索引及距離
    nearest_articles_indices = indices[0]
    nearest_articles_distances = distances[0]
    predicted_sentences = 0
    # 將最相近的五篇文章的刑期加總，並計算平均值作為預測刑期
    for i, index in enumerate(nearest_articles_indices):
        predicted_sentences += train_sentences[index]
    predicted_sentence = float(predicted_sentences) / k

    predicted_sentences_all.append(predicted_sentence)

In [ ]:
mse_linear = mean_squared_error(test_sentences, predicted_sentences_all)
rmse_linear = mean_squared_error(test_sentences, predicted_sentences_all, squared=False)
mae_linear = mean_absolute_error(test_sentences, predicted_sentences_all)
print('\n==================================================')
print("Linear Regression - MSE:", mse_linear)
print("Linear Regression - RMSE:", rmse_linear)
print("Linear Regression - MAE:", mae_linear)


Linear Regression - MSE: 16297.42162368
Linear Regression - RMSE: 127.66135524770212
Linear Regression - MAE: 87.09752


## k = 300

In [ ]:
k = 300
knn_model_SBERT = NearestNeighbors(n_neighbors=k, metric='cosine')
knn_model_SBERT.fit(train_embeddings)

NearestNeighbors(metric='cosine', n_neighbors=300)

In [ ]:
predicted_sentences_all = []

for test_embedding in test_embeddings:
    test_embedding = test_embedding.reshape(1, -1)
    distances, indices = knn_model_SBERT.kneighbors(test_embedding)
    # 取得最相近的五篇文章的索引及距離
    nearest_articles_indices = indices[0]
    nearest_articles_distances = distances[0]
    predicted_sentences = 0
    # 將最相近的五篇文章的刑期加總，並計算平均值作為預測刑期
    for i, index in enumerate(nearest_articles_indices):
        predicted_sentences += train_sentences[index]
    predicted_sentence = float(predicted_sentences) / k

    predicted_sentences_all.append(predicted_sentence)
mse_linear = mean_squared_error(test_sentences, predicted_sentences_all)
rmse_linear = mean_squared_error(test_sentences, predicted_sentences_all, squared=False)
mae_linear = mean_absolute_error(test_sentences, predicted_sentences_all)
print('\n==================================================')
print("Linear Regression - MSE:", mse_linear)
print("Linear Regression - RMSE:", rmse_linear)
print("Linear Regression - MAE:", mae_linear)


Linear Regression - MSE: 16345.296069555556
Linear Regression - RMSE: 127.8487233786695
Linear Regression - MAE: 87.94200000000002


## k = 400

In [ ]:
k = 400
knn_model_SBERT = NearestNeighbors(n_neighbors=k, metric='cosine')
knn_model_SBERT.fit(train_embeddings)

NearestNeighbors(metric='cosine', n_neighbors=400)

In [ ]:
predicted_sentences_all = []

for test_embedding in test_embeddings:
    test_embedding = test_embedding.reshape(1, -1)
    distances, indices = knn_model_SBERT.kneighbors(test_embedding)
    # 取得最相近的五篇文章的索引及距離
    nearest_articles_indices = indices[0]
    nearest_articles_distances = distances[0]
    predicted_sentences = 0
    # 將最相近的五篇文章的刑期加總，並計算平均值作為預測刑期
    for i, index in enumerate(nearest_articles_indices):
        predicted_sentences += train_sentences[index]
    predicted_sentence = float(predicted_sentences) / k

    predicted_sentences_all.append(predicted_sentence)
mse_linear = mean_squared_error(test_sentences, predicted_sentences_all)
rmse_linear = mean_squared_error(test_sentences, predicted_sentences_all, squared=False)
mae_linear = mean_absolute_error(test_sentences, predicted_sentences_all)
print('\n==================================================')
print("Linear Regression - MSE:", mse_linear)
print("Linear Regression - RMSE:", rmse_linear)
print("Linear Regression - MAE:", mae_linear)


Linear Regression - MSE: 16361.134653437499
Linear Regression - RMSE: 127.91065105548286
Linear Regression - MAE: 89.380425


# 平均

In [ ]:
avg_sentence = sum(train_sentences)/len(train_sentences)

In [ ]:
avg_sentence

187.724

In [ ]:
avg_sentences = [avg_sentence]*len(test_sentences)

In [ ]:
mse_linear = mean_squared_error(test_sentences, avg_sentences)
rmse_linear = mean_squared_error(test_sentences, avg_sentences, squared=False)
mae_linear = mean_absolute_error(test_sentences, avg_sentences)

In [ ]:
print('\n==================================================')
print("Linear Regression - MSE:", mse_linear)
print("Linear Regression - RMSE:", rmse_linear)
print("Linear Regression - MAE:", mae_linear)


Linear Regression - MSE: 16325.817136000003
Linear Regression - RMSE: 127.77252105206347
Linear Regression - MAE: 89.10199999999999
